# Parquet For Data Processing

This is a demo of parquet data format and its capabilities for big data processing.
For this demo, we will be using pandas, sqlite, pyarrow and pyspark libraries to demonstrate the parquet capabilities.
The dataset that we will use is an sqlite dump of [wikibooks](https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset) from kaggle. It contains 270K chapters of wikibooks in 12 languages, but we will concentrate on the English version. To access this dataset you need to setup kaggle account and download your [kaggle.json file before proceeding](https://www.kaggle.com/docs/api#authentication).

- pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools in Python.
- sqlite is an embedded SQL database engine, that uses the more traditional [B-Tree data-format](https://www.sqlite.org/fileformat2.html) for storage on disk 
- pyarrow is Python API of the [Apache Arrow](https://arrow.apache.org/) framework that defines an in-memory data representation and can read/write parquet, including conversion to pandas. There are alternatives like [fastparquet](https://pypi.org/project/fastparquet/) which can also be explored.
- pyspark is a Python API to the [Apache Spark Engine](https://spark.apache.org/), interfaces Python commands with a Java/Scala execution core, and thereby gives Python programmers access to the Parquet format as parquet is natively supported in spark.

In [5]:
import sqlite3
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
from IPython.display import display
from ipywidgets import FileUpload
import os
import shutil

# Creating Parquet dataset from sqlite
We will fetch the sqlite dataset and convert and store each table into parquet files. We can then compare the on-disk sizes to get an idea of how efficient parquet is. Note that dataset is about 1.8G so might take a while to download depending on your network speed.

## Setting up kaggle token and downloading the dataset

In [3]:
# Download your kaggle credentials file from kaggle and supply it here, only necessary if you have not yet setup your kaggle credentials
# in the .kaggle folder in your home dir
# Path to the .kaggle directory
kaggle_dir = os.path.expanduser('~/.kaggle')
kaggle_file_path = os.path.join(kaggle_dir, 'kaggle.json')

# Function to check and prompt for file upload
def check_and_prompt_for_upload():
    if not os.path.isfile(kaggle_file_path):
        print("kaggle.json file not found. Please upload the file.")
        upload = FileUpload(accept='application/json', multiple=False)
        display(upload)
        return upload
    else:
        print("kaggle.json file already exists in the '~/.kaggle' directory.")
        return None

# Adjusted function to process the uploaded file based on the provided structure and set permissions
def process_uploaded_file(upload_widget):
    # Ensure the .kaggle directory exists
    os.makedirs(kaggle_dir, exist_ok=True)
    
    if upload_widget:
        # Assuming the first item in the tuple is the file info dictionary
        file_info = upload_widget.value[0]  # Extract the file details from the tuple
        
        content = file_info['content']
        with open(kaggle_file_path, 'wb') as f:
            f.write(content)
        print(f"'{file_info['name']}' has been moved to '{kaggle_dir}'.")

        # Set file permissions to 600
        os.chmod(kaggle_file_path, 0o600)
        print(f"Permissions for '{file_info['name']}' set to 600.")

upload = check_and_prompt_for_upload()

kaggle.json file not found. Please upload the file.


FileUpload(value=(), accept='application/json', description='Upload')

In [4]:
try:
    if upload.value:
        process_uploaded_file(upload)
except NameError:
    print("Upload widget not displayed or file already exists.")

'kaggle.json' has been moved to '/home/ydatta/.kaggle'.
Permissions for 'kaggle.json' set to 600.


In [5]:
import kaggle

In [6]:
!kaggle datasets download -d dhruvildave/wikibooks-dataset

100%|██████████████████████████████████████| 1.82G/1.82G [11:32<00:00, 3.35MB/s]
100%|██████████████████████████████████████| 1.82G/1.82G [11:32<00:00, 2.83MB/s]


In [8]:
from zipfile import ZipFile
file_name = 'wikibooks-dataset.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


## Convert data to parquet files

In [15]:
# Path to the SQLite database file
sqlite_file = 'wikibooks.sqlite'

# Get the size of the SQLite database file
sqlite_file_size_bytes = os.path.getsize(sqlite_file)
# Convert the size from bytes to megabytes (MB)
sqlite_file_size_mb = sqlite_file_size_bytes / (1024 ** 2)

# Establish a connection to the SQLite database
conn = sqlite3.connect(sqlite_file)

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query to retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
table_names = cursor.fetchall()

# Initialize a variable to hold the sum of the sizes of the Parquet files
sum_parquet_files_size_bytes = 0

# Iterate over the table names
for table_name in table_names:
    table_name = table_name[0]  # Extract the table name from the tuple

    file_name = f"{table_name}.parquet"
    
    # Check if the Parquet file already exists
    if os.path.exists(file_name):
        print(f"File '{file_name}' already exists. Skipping...")
        sum_parquet_files_size_bytes += os.path.getsize(file_name)
        continue
    
    # Fetch all the data from the table
    cursor.execute(f"SELECT * FROM {table_name};")
    table_data = cursor.fetchall()

    # Fetch the column names
    cursor.execute(f"PRAGMA table_info({table_name});")
    column_names = cursor.fetchall()
    column_names = [column[1] for column in column_names]

    # Create a pandas DataFrame from the fetched data
    df = pd.DataFrame(table_data, columns=column_names)

    # Convert to arrow format
    table = pa.Table.from_pandas(df)
    # Save as Parquet file
    pq.write_table(table, file_name, row_group_size=10000)

    # Can also write df to parquet file directly but less flexible
    # Save the DataFrame as a Parquet file
    # df.to_parquet(file_name, index=False)
    sum_parquet_files_size_bytes += os.path.getsize(file_name)

    print(f"Table '{table_name}' saved as '{file_name}'")

Table 'pl' saved as 'pl.parquet'
Table 'hu' saved as 'hu.parquet'
Table 'he' saved as 'he.parquet'
Table 'nl' saved as 'nl.parquet'
Table 'ja' saved as 'ja.parquet'
Table 'ru' saved as 'ru.parquet'
Table 'it' saved as 'it.parquet'
Table 'en' saved as 'en.parquet'
Table 'es' saved as 'es.parquet'
Table 'pt' saved as 'pt.parquet'
Table 'de' saved as 'de.parquet'
Table 'fr' saved as 'fr.parquet'


## Space savings

In [3]:
# Convert the sum of the sizes of the Parquet files from bytes to megabytes (MB)
sum_parquet_files_size_mb = sum_parquet_files_size_bytes / (1024 ** 2)

# Calculate the percentage of space saved
space_savings_percentage = (1 - (sum_parquet_files_size_mb / sqlite_file_size_mb)) * 100

# Print the size of the SQLite database file in MB
print(f"Size of SQLite database file: {sqlite_file_size_mb:.2f} MB")

# Print the sum of the sizes of the Parquet files in MB
print(f"Sum of sizes of Parquet files: {sum_parquet_files_size_mb:.2f} MB")

# Print the percentage of space saved
print(f"Percentage of space saved by converting to Parquet: {space_savings_percentage:.2f}%")

NameError: name 'sum_parquet_files_size_bytes' is not defined

# Inspecting Parquet Data Format

As mentioned previously, parquet has a specific way of storing the columnar data that speeds up subsequent queries. One of the crucial aspects is the metadata for each column and organization of Column data into RowGroups. Here we will see how that looks like and what all statistics are pre-generated by parquet and stored along-side data to speed up queries.

In [6]:
# Read the English dataset
en_file =  "en.parquet"
parquet_file = pq.ParquetFile(en_file)

# Inspect file metadata
metadata = parquet_file.metadata
metadata

  created_by: parquet-cpp-arrow version 15.0.0
  num_columns: 5
  num_rows: 86736
  num_row_groups: 9
  format_version: 2.6
  serialized_size: 21408

In [25]:
# Inspect metadata of a Row Group
metadata.row_group(0).column(0)

  file_offset: 325052
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 10000
  path_in_schema: title
  is_stats_set: True
  statistics:
      has_min_max: True
      min: Wikibooks: .NET Development Foundation/AllInOne
      max: Wikibooks: Þe ettbære Garden/S
      null_count: 0
      distinct_count: None
      num_values: 10000
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 4
  data_page_offset: 307407
  total_compressed_size: 325048
  total_uncompressed_size: 617999

- There are a total of 10000 rows in the RowGroup as expected because that is what we set when writing the parquet file.
- Parquet is storing statistics for each column with min and max values, which is useful for eliminating row groups while reading.

In [ ]:
print(f"There are {parquet_file.num_row_groups} RowGroups in {en_file} file")

In [26]:
# Inspect the data schema
parquet_file.schema

required group field_id=-1 schema {
  optional binary field_id=-1 title (String);
  optional binary field_id=-1 url (String);
  optional binary field_id=-1 abstract (String);
  optional binary field_id=-1 body_text (String);
  optional binary field_id=-1 body_html (String);
}

In [7]:
# It is possible to read invidual row groups, since RowGroup offsets are maintained by parquet file
rg = parquet_file.read_row_group(3)
print(rg.to_string())
rg.to_pandas()

pyarrow.Table
title: string
url: string
abstract: string
body_text: string
body_html: string


,title,url,abstract,body_text,body_html
0,Wikibooks: Social Statistics/Chapter 2,https://en.wikibooks.org/wiki/Social_Statistic...,__NOTOC__,Linear Regression Models[edit | edit source]\n...,"<div class=""mw-parser-output""><h1><span class=..."
1,Wikibooks: IB Chemistry,https://en.wikibooks.org/wiki/IB_Chemistry,__NOTOC__ __NOEDITSECTION__,Standard Level Chapters\nThe last cohort of st...,"<div class=""mw-parser-output""><h2><span class=..."
2,Wikibooks: Field Guide/Animal Tracks/Raccoon,https://en.wikibooks.org/wiki/Field_Guide/Anim...,,Raccoon (Procyon lotor)\n\n\n\t\t\n\t\t\t\n\t\...,"<div class=""mw-parser-output""><div style=""bord..."
3,Wikibooks: Lua in SpringRTS/Variables and Cons...,https://en.wikibooks.org/wiki/Lua_in_SpringRTS...,Here follows global constants and variables th...,Here follows global constants and variables th...,"<div class=""mw-parser-output""><p>Here follows ..."
4,Wikibooks: Solutions To Mathematics Textbooks/...,https://en.wikibooks.org/wiki/Solutions_To_Mat...,=Chapter 6=,Contents\n\n1 Chapter 6\n\n1.1 7\n\n1.1.1 a\n1...,"<div class=""mw-parser-output""><div id=""toc"" cl..."
...,...,...,...,...,...
9995,Wikibooks: Radiation Oncology/RTOG Trials,https://en.wikibooks.org/wiki/Radiation_Oncolo...,Overview of RTOG Trials,Front Page: Radiation Oncology | RTOG Trials |...,"<div class=""mw-parser-output""><table width=""75..."
9996,Wikibooks: AP Biology/LABORATORY 11. Animal Be...,https://en.wikibooks.org/wiki/AP_Biology/LABOR...,This is a lab performed by AP Biology students...,This is a lab performed by AP Biology students...,"<div class=""mw-parser-output""><p>This is a lab..."
9997,Wikibooks: Web App Development with Google App...,https://en.wikibooks.org/wiki/Web_App_Developm...,== Create a new script ==,Create a new script[edit | edit source]\nYou c...,"<div class=""mw-parser-output""><h2><span class=..."
9998,Wikibooks: Artificial Intelligence/Search/Dijk...,https://en.wikibooks.org/wiki/Artificial_Intel...,==Overview==,Contents\n\n1 Overview\n2 Description of the A...,"<div class=""mw-parser-output""><div id=""toc"" cl..."
